In [88]:
pip install pyopenssl --upgrade

  Using cached cryptography-38.0.1-cp36-abi3-macosx_10_10_x86_64.whl (2.8 MB)
  Attempting uninstall: cryptography
    Found existing installation: cryptography 35.0.0
    Uninstalling cryptography-35.0.0:
      Successfully uninstalled cryptography-35.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
snowflake-connector-python 2.8.0 requires cryptography<37.0.0,>=3.1.0, but you have cryptography 38.0.1 which is incompatible.
great-expectations 0.14.13 requires jinja2<3.1.0,>=2.10, but you have jinja2 3.1.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [91]:
!python -m OpenSSL.debug

pyOpenSSL: 22.1.0
cryptography: 38.0.1
cffi: 1.15.1
cryptography's compiled against OpenSSL: OpenSSL 3.0.5 5 Jul 2022
cryptography's linked OpenSSL: OpenSSL 3.0.5 5 Jul 2022
Python's OpenSSL: OpenSSL 1.1.1q  5 Jul 2022
Python executable: /Users/franciscojavierarceo/GitHub/feast/venv/bin/python
Python version: 3.8.14 (default, Sep  6 2022, 23:26:50) 
[Clang 13.1.6 (clang-1316.0.21.2.5)]
Platform: darwin
sys.path: ['/Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/dqm_repo/feature_repo', '/usr/local/Cellar/python@3.8/3.8.14/Frameworks/Python.framework/Versions/3.8/lib/python38.zip', '/usr/local/Cellar/python@3.8/3.8.14/Frameworks/Python.framework/Versions/3.8/lib/python3.8', '/usr/local/Cellar/python@3.8/3.8.14/Frameworks/Python.framework/Versions/3.8/lib/python3.8/lib-dynload', '/Users/franciscojavierarceo/GitHub/feast/venv/lib/python3.8/site-packages', '/Users/franciscojavierarceo/GitHub/feast/sdk/python']


In [ ]:
! pip install jinja2==3.0.3

  Using cached Jinja2-3.0.3-py3-none-any.whl (133 kB)
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.1.2
    Uninstalling Jinja2-3.1.2:
      Successfully uninstalled Jinja2-3.1.2


In [1]:
cd /Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/

/Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring


In [2]:
! rm -r dqm_repo

In [3]:
!feast init dqm_repo


Creating a new Feast repository in /Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/dqm_repo.



In [4]:
cd /Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/dqm_repo/feature_repo

/Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/dqm_repo/feature_repo


In [5]:
x = """
from datetime import timedelta
from feast import Entity
taxi_entity = Entity(name='taxi', join_keys=['taxi_id'])
from feast import FileSource, Field
from feast.data_format import ParquetFormat
from feast.types import Float32, Float64, Int64

batch_source = FileSource(
    timestamp_field="day",
    # using parquet file that we created on previous step
    path="/Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/trips_stats.parquet",  
    file_format=ParquetFormat()
)


from feast import BatchFeatureView

trips_stats_fv = BatchFeatureView(
    name='trip_stats',
    entities=[taxi_entity],
    ttl=timedelta(seconds=86400),
    schema=[
        Field(name="total_miles_travelled", dtype=Float64),
        Field(name="total_trip_seconds", dtype=Float64),
        Field(name="total_earned", dtype=Float64),
        Field(name="trip_count", dtype=Int64),

    ],
    source=batch_source,
    online=True,
    tags={"production": "True"},
    owner='test1@gmail.com',
)

import pyarrow.parquet
import pandas as pd

from feast import FeatureView, Entity, FeatureStore, Field, BatchFeatureView
from feast.types import Float64, Int64
from feast.value_type import ValueType
from feast.data_format import ParquetFormat
from feast.on_demand_feature_view import on_demand_feature_view
from feast.infra.offline_stores.file_source import FileSource
from feast.infra.offline_stores.file import SavedDatasetFileStorage
from datetime import timedelta

@on_demand_feature_view(
    schema=[
        Field(name="avg_fare", dtype=Float64),
        Field(name="avg_speed", dtype=Float64),
        Field(name="avg_trip_seconds", dtype=Float64),
        Field(name="earned_per_hour", dtype=Float64),
    ],
    sources=[
      trips_stats_fv,
    ]
)
def on_demand_stats(inp):
    out = pd.DataFrame()
    out["avg_fare"] = inp["total_earned"] / inp["trip_count"]
    out["avg_speed"] = 3600 * inp["total_miles_travelled"] / inp["total_trip_seconds"]
    out["avg_trip_seconds"] = inp["total_trip_seconds"] / inp["trip_count"]
    out["earned_per_hour"] = 3600 * inp["total_earned"] / inp["total_trip_seconds"]
    return out

"""

In [6]:
f = open('/Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/dqm_repo/feature_repo/dqm_example.py', 'w')
f.write(x)
f.close()

In [7]:
! feast apply

/Users/franciscojavierarceo/GitHub/feast/sdk/python/feast/batch_feature_view.py:72: RuntimeWarning: Batch feature views are experimental features in alpha development. Some functionality may still be unstable so functionality can change in the future.
  warnings.warn(
/Users/franciscojavierarceo/GitHub/feast/sdk/python/feast/feature_store.py:563: RuntimeWarning: On demand feature view is an experimental feature. This API is stable, but the functionality does not scale well for offline retrieval
  warnings.warn(
/Users/franciscojavierarceo/GitHub/feast/sdk/python/feast/infra/offline_stores/file_source.py:161: FutureWarning: 'ParquetDataset.schema' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the ParquetDataset, and then use the '.schema' attribute instead (which will return an Arrow schema instead of a Parquet schema).
  schema = ParquetDataset(path).schema.to_arrow_schema()
Created entity taxi

# Now the DQM piece

In [8]:
import pyarrow.parquet
taxi_ids = pyarrow.parquet.read_table("/Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/entities.parquet").to_pandas()

In [9]:
import pandas as pd
timestamps = pd.DataFrame()
timestamps["event_timestamp"] = pd.date_range("2019-06-01", "2019-07-01", freq='D')

In [10]:
entity_df = pd.merge(taxi_ids, timestamps, how='cross')
entity_df.head()

,taxi_id,event_timestamp
0,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-01
1,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-02
2,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-03
3,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-04
4,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2019-06-05


In [11]:
from feast import FeatureStore
store = FeatureStore("/Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/dqm_repo/feature_repo/")

In [12]:
job = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "trip_stats:total_miles_travelled",
        "trip_stats:total_trip_seconds",
        "trip_stats:total_earned",
        "trip_stats:trip_count",
        "on_demand_stats:avg_fare",
        "on_demand_stats:avg_trip_seconds",
        "on_demand_stats:avg_speed",
        "on_demand_stats:earned_per_hour",
    ]
)

In [13]:
from feast.infra.offline_stores.file import SavedDatasetFileStorage

try:
    thing = store.create_saved_dataset(
        from_=job,
        name='my_training_ds',
        storage=SavedDatasetFileStorage(path='/Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/dqm_repo/feature_repo/my_training_ds.parquet')
    )
except:
    pass

/Users/franciscojavierarceo/GitHub/feast/sdk/python/feast/feature_store.py:1182: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [14]:
store.apply(thing)

# Now setting up the profiler

In [15]:
store.registry.list_saved_datasets(project='dqm_repo')[0]

<SavedDataset(name = my_training_ds, features = ['trip_stats:total_miles_travelled', 'trip_stats:total_trip_seconds', 'trip_stats:total_earned', 'trip_stats:trip_count', 'on_demand_stats:avg_fare', 'on_demand_stats:avg_trip_seconds', 'on_demand_stats:avg_speed', 'on_demand_stats:earned_per_hour'], join_keys = ['taxi_id'], storage = <feast.infra.offline_stores.file_source.SavedDatasetFileStorage object at 0x137d029d0>, full_feature_names = False, tags = {}, feature_service_name = None, _retrieval_job = None, created_timestamp = 2022-10-08 13:40:13.884689, min_event_timestamp = 2019-06-01 00:00:00, max_event_timestamp = 2019-07-01 00:00:00)>

In [16]:
import numpy as np

from feast.dqm.profilers.ge_profiler import ge_profiler

from great_expectations.core.expectation_suite import ExpectationSuite
from great_expectations.dataset import PandasDataset


/Users/franciscojavierarceo/GitHub/feast/venv/lib/python3.8/site-packages/botocore/httpsession.py:28: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
  from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext


In [17]:
ds = store.get_saved_dataset('my_training_ds')
ds.to_df()

/Users/franciscojavierarceo/GitHub/feast/sdk/python/feast/feature_store.py:1235: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


,taxi_id,event_timestamp,avg_trip_seconds,trip_count,total_miles_travelled,earned_per_hour,avg_speed,total_earned,avg_fare,total_trip_seconds
0,d13c5aaa066f94b4927779ed24cd313b0c686f03407095...,2019-06-01 00:00:00+00:00,2010.000000,8,69.50,45.559701,15.559701,203.50,25.437500,16080
1,33164e16dd29b1c58cd15cce31df4bfcb75d9903cb66de...,2019-06-01 00:00:00+00:00,1476.000000,5,15.80,36.219512,7.707317,74.25,14.850000,7380
2,226fe0b00be42932bdff81bc0b318b883bfbf15dd48093...,2019-06-01 00:00:00+00:00,1270.000000,6,38.50,54.212598,18.188976,114.75,19.125000,7620
3,5a5bed1b5ced617d0594007d591f10bbbca354d50b19ca...,2019-06-01 00:00:00+00:00,1415.000000,4,20.22,45.000000,12.860777,70.75,17.687500,5660
4,b7f7dbb452c0fb980a0f2050a146147c1006fe5f34e3b0...,2019-06-01 00:00:00+00:00,1395.600000,5,34.49,53.783319,17.793637,104.25,20.850000,6978
...,...,...,...,...,...,...,...,...,...,...
119803,961263722c1beadafef2355412d672acac35e4054f6aaa...,2019-07-01 00:00:00+00:00,1205.000000,4,36.98,76.369295,27.619917,102.25,25.562500,4820
119804,8b07f9156e568a37d362463c84dbd1118b4eeb753bae50...,2019-07-01 00:00:00+00:00,692.727273,11,29.00,52.677165,13.700787,111.50,10.136364,7620
119805,a112879f10892d5c698ce150af17aa28615b6d005ca749...,2019-07-01 00:00:00+00:00,588.750000,16,31.00,54.649682,11.847134,143.00,8.937500,9420
119806,68fe14b9fc2d53de5ac349d47f80f43fea895e201a31e3...,2019-07-01 00:00:00+00:00,1647.000000,3,37.86,73.770492,27.584699,101.25,33.750000,4941


In [18]:
import numpy as np

from great_expectations.core.expectation_suite import ExpectationSuite
from great_expectations.dataset import PandasDataset
from feast.dqm.profilers.ge_profiler import ge_profiler

In [19]:
DELTA = 0.1  # controlling allowed window in fraction of the value on scale [0, 1]

@ge_profiler
def stats_profiler(ds: PandasDataset) -> ExpectationSuite:    
    # simple checks on data consistency
    ds.expect_column_values_to_be_between(
        "avg_speed",
        min_value=0,
        max_value=60,
        mostly=0.99  # allow some outliers
    )

    ds.expect_column_values_to_be_between(
        "total_miles_travelled",
        min_value=0,
        max_value=500,
        mostly=0.99  # allow some outliers
    )

    # expectation of means based on observed values
    observed_mean = ds.trip_count.mean()
    ds.expect_column_mean_to_be_between("trip_count",
                                        min_value=observed_mean * (1 - DELTA),
                                        max_value=observed_mean * (1 + DELTA))

    observed_mean = ds.earned_per_hour.mean()
    ds.expect_column_mean_to_be_between("earned_per_hour",
                                        min_value=observed_mean * (1 - DELTA),
                                        max_value=observed_mean * (1 + DELTA))

    # expectation of quantiles
    qs = [0.5, 0.75, 0.9, 0.95]
    observed_quantiles = ds.avg_fare.quantile(qs)

    ds.expect_column_quantile_values_to_be_between(
        "avg_fare",
        quantile_ranges={
            "quantiles": qs,
            "value_ranges": [[None, max_value] for max_value in observed_quantiles]
        })

    # Note that the default is TRUE!
    # return ds.get_expectation_suite(discard_failed_expectations=False)
    return ds.get_expectation_suite()


@ge_profiler
def stats_profiler_fail(ds: PandasDataset) -> ExpectationSuite:
    # This should fail
    ds.expect_column_to_exist("this is a test")
    return ds.get_expectation_suite(discard_failed_expectations=False)

@ge_profiler
def stats_profiler_pass(ds: PandasDataset) -> ExpectationSuite:
    # This should fail
    ds.expect_column_to_exist("trip_count")
    return ds.get_expectation_suite(discard_failed_expectations=False)

In [20]:
prof = ds.get_profile(profiler=stats_profiler)
prof_fail = ds.get_profile(profiler=stats_profiler_fail)
prof_pass = ds.get_profile(profiler=stats_profiler_pass)

# Verify that we are getting all of the expectations we intended

In [21]:
assert len(prof.expectation_suite['expectations']) == 5
assert len(prof_fail.expectation_suite['expectations']) == 1
assert len(prof_pass.expectation_suite['expectations']) == 1

In [22]:
validation_reference = ds.as_reference(name='my_validator', profiler=stats_profiler)
validation_reference_fail = ds.as_reference(name='my_validator', profiler=stats_profiler_fail)
validation_reference_pass = ds.as_reference(name='my_validator', profiler=stats_profiler_pass)

In [23]:
from feast.dqm.errors import ValidationFailed

In [24]:
timestamps = pd.DataFrame()
timestamps["event_timestamp"] = pd.date_range("2020-12-01", "2020-12-07", freq='D')
entity_df = pd.merge(taxi_ids, timestamps, how='cross')
entity_df.head()

,taxi_id,event_timestamp
0,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-01
1,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-02
2,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-03
3,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-04
4,91d5288487e87c5917b813ba6f75ab1c3a9749af906a2d...,2020-12-05


In [25]:
job = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "trip_stats:total_miles_travelled",
        "trip_stats:total_trip_seconds",
        "trip_stats:total_earned",
        "trip_stats:trip_count",
        "on_demand_stats:avg_fare",
        "on_demand_stats:avg_trip_seconds",
        "on_demand_stats:avg_speed",
        "on_demand_stats:earned_per_hour",
    ]
)

In [26]:
try:
    df = job.to_df(validation_reference=validation_reference_pass)
    print('validation passed')
except ValidationFailed as exc:
    print('validation failed\n', exc.validation_report)

/Users/franciscojavierarceo/GitHub/feast/sdk/python/feast/infra/offline_stores/offline_store.py:90: RuntimeWarning: Dataset validation is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


validation passed


In [27]:
try:
    df = job.to_df(validation_reference=validation_reference_fail)
except ValidationFailed as exc:
    print('validation failed\n', exc.validation_report)

/Users/franciscojavierarceo/GitHub/feast/sdk/python/feast/infra/offline_stores/offline_store.py:90: RuntimeWarning: Dataset validation is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


validation failed
 [
  {
    "meta": {},
    "result": {},
    "exception_info": {
      "raised_exception": false,
      "exception_message": null,
      "exception_traceback": null
    },
    "expectation_config": {
      "expectation_type": "expect_column_to_exist",
      "meta": {},
      "kwargs": {
        "column": "this is a test",
        "result_format": "COMPLETE"
      }
    },
    "success": false
  }
]


In [28]:
try:
    df = job.to_df(validation_reference=validation_reference)
except ValidationFailed as exc:
    print('validation failed\n', exc.validation_report)

/Users/franciscojavierarceo/GitHub/feast/sdk/python/feast/infra/offline_stores/offline_store.py:90: RuntimeWarning: Dataset validation is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


validation failed
 [
  {
    "meta": {},
    "result": {
      "observed_value": 6.692920555429092,
      "element_count": 4393,
      "missing_count": null,
      "missing_percent": null
    },
    "exception_info": {
      "raised_exception": false,
      "exception_message": null,
      "exception_traceback": null
    },
    "expectation_config": {
      "expectation_type": "expect_column_mean_to_be_between",
      "meta": {},
      "kwargs": {
        "column": "trip_count",
        "min_value": 10.387244591346153,
        "max_value": 12.695521167200855,
        "result_format": "COMPLETE"
      }
    },
    "success": false
  },
  {
    "meta": {},
    "result": {
      "observed_value": 68.99268345164135,
      "element_count": 4393,
      "missing_count": null,
      "missing_percent": null
    },
    "exception_info": {
      "raised_exception": false,
      "exception_message": null,
      "exception_traceback": null
    },
    "expectation_config": {
      "expectation_type"

# END